In [6]:
import plotly.express as px
import pandas as pd 

In [7]:
df = pd.read_csv("clean_datasets/rte_2014_2024.csv" : "sum",index_col=False)
display(df.head())

/tmp/ipykernel_15296/194515446.py:1: DtypeWarning: Columns (18,19,20,21,22,26,27,30,31,33,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("clean_datasets/rte_2014_2024.csv",index_col=False)


,Périmètre,Nature,Date,Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,...,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore,Type de jour TEMPO
0,France,Données définitives,2014-01-01,00:00,64660.0,63200,63200.0,454.0,0.0,2303.0,...,2125,693.0,460,165.0,179.0,NaN,NaN,NaN,NaN,NaN
1,France,Données définitives,2014-01-01,00:30,63494.0,62100,61500.0,281.0,0.0,2367.0,...,2450,495.0,592,170.0,181.0,NaN,NaN,NaN,NaN,NaN
2,France,Données définitives,2014-01-01,01:00,61362.0,59900,59900.0,281.0,0.0,2188.0,...,2040,76.0,591,173.0,181.0,NaN,NaN,NaN,NaN,NaN
3,France,Données définitives,2014-01-01,01:30,61217.0,60600,60300.0,281.0,0.0,2190.0,...,1849,0.0,595,174.0,180.0,NaN,NaN,NaN,NaN,NaN
4,France,Données définitives,2014-01-01,02:00,60748.0,59900,60200.0,281.0,0.0,2187.0,...,1592,0.0,597,174.0,180.0,NaN,NaN,NaN,NaN,NaN


In [8]:
# Suppression 2014 car pas de jour temp associé
df["Date"]=pd.to_datetime(df["Date"])
df = df.loc[df["Date"].dt.year>2014,:]
# Suppression des colonnes détails d"energie
columns_to_keep = [
    "Nature" : "sum",
    "Date" : "sum" : "sum" : "sum",
    "Consommation",
    "Prévision J-1",
    "Prévision J",
    "Fioul",
    "Charbon",
    "Gaz",
    "Nucléaire",
    "Eolien",
    "Solaire",
    "Hydraulique",
    "Pompage",
    "Bioénergies",
    "Ech. physiques",
    "Taux de Co2",
    "Ech. comm. Angleterre",
    "Ech. comm. Espagne",
    "Ech. comm. Italie",
    "Ech. comm. Suisse",
    "Ech. comm. Allemagne-Belgique",
    "Type de jour TEMPO",
]
df=df[columns_to_keep]
# Concat des echange commerciaux
list_col_to_sum = [
    "Ech. comm. Angleterre",
    "Ech. comm. Espagne",
    "Ech. comm. Italie",
    "Ech. comm. Suisse",
    "Ech. comm. Allemagne-Belgique"
]
for col in list_col_to_sum:
    df[col] = df[col].replace("ND", None).astype(float)

df["Ech_comm"] = df[list_col_to_sum].sum(axis=1)
df.drop(list_col_to_sum,axis=1, inplace= True)
df.head()

,Nature,Date,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,Nucléaire,Eolien,Solaire,Hydraulique,Pompage,Bioénergies,Ech. physiques,Taux de Co2,Stockage batterie,Déstockage batterie,Type de jour TEMPO,Ech_comm
17520,Données définitives,2015-01-01,74494.0,72900,73700.0,269.0,2487.0,3476.0,57982.0,1268.0,0.0,9321.0,-25.0,938.0,-1221.0,62.0,NaN,NaN,BLANC,-1035.0
17521,Données définitives,2015-01-01,73481.0,71600,72400.0,319.0,2087.0,2691.0,57856.0,1343.0,0.0,9664.0,-20.0,896.0,-1356.0,54.0,NaN,NaN,BLANC,-1035.0
17522,Données définitives,2015-01-01,71506.0,69900,70600.0,320.0,1716.0,2436.0,57787.0,1356.0,0.0,8479.0,-20.0,893.0,-1460.0,49.0,NaN,NaN,BLANC,-1293.0
17523,Données définitives,2015-01-01,71505.0,70600,71300.0,320.0,1589.0,2440.0,58156.0,1412.0,0.0,8293.0,-21.0,904.0,-1586.0,47.0,NaN,NaN,BLANC,-1293.0
17524,Données définitives,2015-01-01,71134.0,70500,71100.0,320.0,1490.0,2441.0,57974.0,1447.0,0.0,8103.0,-21.0,902.0,-1521.0,46.0,NaN,NaN,BLANC,-984.0


In [32]:
groupby = [
    "Date",
    "Type de jour TEMPO"
]
column_aggregations = {
    "Consommation" : "sum",
    "Prévision J-1" : "sum",
    "Prévision J" : "sum",
    "Fioul" : "sum",
    "Charbon" : "sum",
    "Gaz" : "sum",
    "Nucléaire" : "sum",
    "Eolien" : "sum",
    "Solaire" : "sum",
    "Hydraulique" : "sum",
    "Pompage" : "sum",
    "Bioénergies" : "sum",
    "Ech. physiques" : "sum",
    "Taux de Co2" : "sum",
    "Ech_comm" : "sum"
}
df_quotidien = df.groupby(groupby).agg(column_aggregations)
for col in set(df_quotidien.columns) - set(groupby):
    df_quotidien[col] = df_quotidien[col] / 2
df_quotidien

,,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,Nucléaire,Eolien,Solaire,Hydraulique,Pompage,Bioénergies,Ech. physiques,Taux de Co2,Ech_comm
Date,Type de jour TEMPO,,,,,,,,,,,,,,,
2015-01-01,BLANC,1613817.0,1569650.0,1603950.0,7666.5,28191.5,56216.0,1388930.5,51127.0,11370.5,158136.5,-26906.5,21712.5,-82623.5,1028.0,-76107.0
2015-01-02,BLANC,1656045.5,1672200.0,1643050.0,7796.5,43440.5,62653.5,1400287.0,78933.0,8297.5,180503.0,-17687.5,21602.5,-129775.5,1196.0,-121458.0
2015-01-03,BLEU,1538261.0,1543750.0,1528950.0,7723.0,34074.0,54482.0,1376450.5,105299.0,5860.5,141242.5,-32067.0,21850.0,-176653.5,1086.0,-168566.0
2015-01-04,BLEU,1413088.0,1436800.0,1407550.0,7713.5,29848.0,54893.5,1328030.0,30061.0,6926.0,145347.5,-43888.0,21477.0,-167324.0,1112.5,-158445.0
2015-01-05,ROUGE,1710599.5,1670450.0,1694950.0,8009.0,63351.5,121532.0,1395291.0,16004.0,9786.5,209115.5,-19182.5,20816.5,-114123.5,1668.0,-105154.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-04,BLEU,1165849.5,1170150.0,1166425.0,2997.0,0.0,17460.0,867263.0,285321.0,76581.5,192826.0,-29510.0,19056.5,-266111.0,318.5,0.0
2024-04-05,BLEU,1113699.5,1122500.0,1101100.0,2998.0,0.0,14497.0,815530.0,232208.5,72847.5,192013.5,-30277.5,19070.5,-204951.5,320.0,0.0
2024-04-06,BLEU,970963.0,957225.0,973500.0,2996.5,0.0,14005.0,649541.0,225106.0,61577.5,184851.0,-31579.0,18938.5,-155174.5,368.0,0.0


In [33]:

fig = px.histogram(df_quotidien, x="Ech_comm", nbins=100, title="Distribution des échanges commerciaux de l'électricité française quotidienne entre 2015 et 2024 (en MJ)")
fig.show()

In [34]:
df_quotidien.to_csv("clean_datasets/rte_agg_daily_2014_2024.csv", index=False)